In [42]:
import csv
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import gurobipy as gp
from gurobipy import GRB

In [3]:
def importClean_5ind(csv_file_path, desired_returns = 'Average Value Weighted Returns -- Monthly'):
    """
    Cette fonction importe des données à partir d'un fichier CSV spécifié, effectue un nettoyage de données,
    et retourne un DataFrame contenant des données filtrées sans tenir compte de la variable T (nombre d'années).

    Args:
        csv_file_path(str): Chemin vers le fichier CSV avec les données brutes.

    Returns:
        DataFrame: DataFrame contenant les données filtrées.
    """

    # Importation
    with open(csv_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        
        # Sauter les 10 premières lignes
        for _ in range(10):
            next(reader)

        # Lire le reste du fichier CSV dans un DataFrame
        df = pd.DataFrame(reader)
        df.columns = df.iloc[1]
        df = df.drop(df.index[1])
        df = df.reset_index(drop=True)

    # Nettoyage
    df = df.rename(columns={df.columns[0]: 'Date'})
    first_row = df[df.iloc[:, 0].str.strip() == desired_returns].index
    df_AEWR_monthly = df.iloc[first_row.values[0] + 2:].reset_index(drop=True)
    last_row = (df_AEWR_monthly['Date'].str.len() != 6).idxmax()
    df_AEWR_monthly = df_AEWR_monthly.iloc[:last_row]
    df_AEWR_monthly['Date'] = pd.to_datetime(df_AEWR_monthly['Date'], format='%Y%m')
    df_AEWR_monthly.iloc[:, 1:] = df_AEWR_monthly.iloc[:, 1:].astype(float)
    df_AEWR_monthly.replace(-99.99, np.nan, inplace=True)
    df_AEWR_monthly.replace(-999, np.nan, inplace=True)
    df_AEWR_monthly.dropna(inplace=True)

    # Retourner le DataFrame complet sans filtre temporel
    df_10Ind = df_AEWR_monthly.reset_index(drop=True)
    
    return df_10Ind

In [4]:
df_48ind = importClean_5ind('/Users/thomasdeconinck/Documents/GitHub/Gestion de Portefeuille Devoir 2/Partie B/Data/48_Industry_Portfolios.CSV', desired_returns='Average Value Weighted Returns -- Monthly')
# Set 'Date' column as the index of the DataFrame
df_48ind.set_index('Date', inplace=True)
df_48ind

/var/folders/gm/cwbp26c51d14t923q1b9pdn00000gn/T/ipykernel_6663/397350783.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_AEWR_monthly.replace(-99.99, np.nan, inplace=True)


1,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,-8.32,-6.92,-3.53,-2.10,-4.74,-7.87,-10.15,-11.07,-3.43,-11.91,...,-3.15,-12.50,-10.09,-8.19,-9.93,-6.60,-10.18,-12.74,-9.05,-14.64
1969-08-01,4.23,4.52,5.44,6.63,3.99,6.42,11.56,9.79,4.77,2.40,...,6.08,5.59,6.38,8.21,8.98,12.76,8.80,15.06,8.89,8.38
1969-09-01,1.59,-0.58,5.28,-0.91,4.12,2.13,0.57,-0.61,-0.52,-2.64,...,1.23,-1.56,0.21,-0.75,-1.35,-2.94,7.08,-3.79,-2.54,-6.67
1969-10-01,16.95,7.97,6.84,11.64,12.42,6.28,13.57,7.59,6.41,6.72,...,3.50,4.89,11.25,6.25,9.03,10.71,5.71,4.54,10.31,3.70
1969-11-01,-1.87,-0.66,-4.91,-2.83,0.61,-8.33,-3.27,-3.97,-2.42,-2.90,...,-1.98,-7.91,-6.83,-4.08,-1.90,-8.99,-4.90,-6.23,-5.91,-2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.84,-4.28,-6.43,-5.13,-2.43,-7.38,-11.17,-6.19,-6.81,-4.41,...,-2.57,-5.15,-2.69,-6.16,-4.45,-3.92,2.57,-10.45,-3.49,-2.72
2023-10-01,-5.83,-4.98,-0.39,-4.34,-4.00,-14.31,2.77,-2.05,-0.05,4.21,...,-3.46,-6.31,-0.90,0.74,-1.71,-2.89,3.49,-9.05,-6.31,-1.77
2023-11-01,-2.29,4.26,4.79,3.90,4.76,5.66,11.08,7.90,4.97,10.63,...,10.43,12.31,7.94,6.96,7.67,13.16,3.93,9.95,13.51,6.64


In [5]:
df_number_firm = importClean_5ind('/Users/thomasdeconinck/Documents/GitHub/Gestion de Portefeuille Devoir 2/Partie B/Data/48_Industry_Portfolios.CSV', desired_returns= 'Number of Firms in Portfolios')
# Set 'Date' column as the index of the DataFrame
df_number_firm.set_index('Date', inplace=True)

df_average_firm_size = importClean_5ind('/Users/thomasdeconinck/Documents/GitHub/Gestion de Portefeuille Devoir 2/Partie B/Data/48_Industry_Portfolios.CSV', desired_returns='Average Firm Size')
# Set 'Date' column as the index of the DataFrame
df_average_firm_size.set_index('Date', inplace=True)

/var/folders/gm/cwbp26c51d14t923q1b9pdn00000gn/T/ipykernel_6663/397350783.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_AEWR_monthly.replace(-99.99, np.nan, inplace=True)


### Market Cap

In [98]:
market_caps = df_average_firm_size.multiply(df_number_firm)
market_caps.dropna(inplace=True)

### BTOM (SUM OF BE/ SUM OF ME)

In [121]:
df_BtoM = importClean_5ind('/Users/thomasdeconinck/Documents/GitHub/Gestion de Portefeuille Devoir 2/Partie B/Data/48_Industry_Portfolios.CSV', 'Sum of BE / Sum of ME')
df_BtoM.set_index('Date', inplace=True)

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)]


df_BtoM = df_BtoM.reset_index(drop=True)
# Supprimer les lignes en excès de df_BtoM
df_BtoM = df_BtoM.iloc[:len(df_48ind)]
# Réattribuer l'index
df_BtoM.index = df_48ind.index

/var/folders/gm/cwbp26c51d14t923q1b9pdn00000gn/T/ipykernel_6663/2970940462.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_AEWR_monthly.replace(-99.99, np.nan, inplace=True)


### MOM

In [8]:
df_mom = df_48ind.rolling(window=12, min_periods=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

df_mom

1,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-1.608333,-1.192500,-0.079167,-0.121667,1.379167,-4.351667,-2.588333,-4.757500,-1.919167,-4.050000,...,-2.296667,-4.505000,-4.432500,-2.108333,-4.271667,-0.481667,-1.880000,-4.616667,-3.060833,-7.690833
1970-07-01,-0.982500,-0.151667,0.701667,0.616667,2.065833,-2.652500,-1.698333,-2.729167,-1.250000,-2.367500,...,-1.479167,-2.764167,-3.231667,-0.444167,-3.360833,0.620833,-0.150000,-2.749167,-1.442500,-6.460833
1970-08-01,-1.360833,-0.497500,0.105833,-0.161667,1.787500,-2.236667,-2.392500,-3.003333,-1.295833,-1.963333,...,-1.340000,-2.526667,-3.212500,-0.702500,-3.523333,0.090000,-0.750833,-3.533333,-1.712500,-5.955000
1970-09-01,-0.289167,-0.032500,0.001667,0.286667,1.486667,-1.895833,-0.841667,-2.345000,-0.720833,-0.746667,...,-1.426667,-1.650000,-1.866667,-0.072500,-2.071667,0.301667,-0.614167,-1.979167,-0.775833,-4.920000
1970-10-01,-1.998333,-0.698333,-0.209167,-0.768333,1.314167,-2.781667,-2.284167,-3.220000,-1.268333,-1.613333,...,-1.936667,-2.470000,-3.435833,-0.665833,-3.153333,-1.025833,-1.920833,-3.140000,-2.250833,-6.836667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.443333,0.130833,0.800833,0.579167,1.386667,0.681667,2.873333,2.539167,1.005833,1.811667,...,1.295000,1.810000,2.147500,1.186667,1.638333,0.859167,0.887500,1.377500,1.430000,2.147500
2023-10-01,-2.074167,-1.222500,0.175000,-0.571667,0.047500,-0.840000,1.854167,1.636667,0.530000,1.255833,...,0.534167,0.715833,0.934167,1.092500,0.615000,-0.496667,0.110000,0.045833,-0.054167,1.101667
2023-11-01,-2.465833,-1.114167,-0.067500,-0.516667,-0.021667,-0.790833,2.170833,0.989167,0.159167,0.682500,...,0.526667,0.895000,1.174167,1.427500,0.755000,0.169167,0.320833,-0.030833,0.355000,1.069167


### Problème de shape dataframe MoM 

In [118]:
df_BtoM.shape

(655, 48)

In [119]:
market_caps.shape

(655, 48)

In [117]:
df_mom.shape

(644, 48)

### Standardiser

In [108]:
# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_caps)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_caps.div(market_caps.sum(axis=1), axis=0)

In [109]:
# Initialisation des données
N = 48  # Nombre d'actifs
T = 54  # Nombre d'observations
gamma = 5  # Coefficient d'aversion au risque CRRA

In [116]:
from scipy.optimize import minimize

# Définir la fonction d'utilité CRRA
def crra_utility(x, gamma):
    if gamma == 1:
        return np.log(x)
    else:
        return (x**(1 - gamma) - 1) / (1 - gamma)
    
# Choisir une valeur initiale pour x
rp = 1.06

# Choisir une valeur pour gamma
gamma = 5

# Utiliser scipy.optimize.minimize pour trouver le maximum de la fonction d'utilité
result = minimize(crra_utility, rp, args=(gamma))

# Imprimer le résultat
max_utility = -result.fun
max_utility

-0.05197658419049489

In [112]:
MC = market_caps  # Capitaux de marché
BM = df_BtoM  # Ratio book-to-market
MOM = df_mom # Momentum
r = df_48ind.shift(-1)   # Rendements des actifs

# Utiliser le maximum de la fonction d'utilité comme une constante dans le modèle Gurobi
max_utility = -result.fun

# Créer le modèle d'optimisation
m = gp.Model("utility_maximization")

# Ajouter les variables de décision pour theta
theta = m.addVars(3, lb=-gp.GRB.INFINITY, name="theta")

# Ajouter la fonction objectif
obj = 0
for t in range(T-1):
    portfolio_return_t = sum((MC.iloc[t, i] * theta[0] + BM.iloc[t, i] * theta[1] + MOM.iloc[t, i] * theta[2]) 
                             * r.iloc[t, i] for i in range(N))
    obj = portfolio_return_t

# Définir la contrainte que l'utilité du portefeuille doit être au moins égale à l'utilité maximale
m.addConstr(obj >= max_utility)

# Optimiser le modèle
m.optimize()

# Créer un DataFrame avec les valeurs optimisées de theta
theta_values = [theta[i].x for i in range(3)]
tilts = pd.DataFrame(theta_values, columns=['Tilt'], index=['MC', 'BM', 'MOM'])

# Afficher le DataFrame
tilts

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 21.6.0 21G72)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x1fc51fbb
Coefficient statistics:
  Matrix range     [5e+01, 1e+06]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 5e-02]
Presolve removed 1 rows and 3 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   3.000000e-06      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  0.000000000e+00


,Tilt
MC,-5.096530e-08
BM,0.000000e+00
MOM,0.000000e+00
